<a href="https://colab.research.google.com/github/mzohaibnasir/intv-p/blob/main/tidbits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# number to words
def num_to_word(n):
    ones = [
        "zero",
        "one",
        "two",
        "three",
        "four",
        "five",
        "six",
        "seven",
        "eight",
        "nine",
    ]
    teens = [
        "ten",
        "eleven",
        "twelve",
        "thirteen",
        "fourteen",
        "fifteen",
        "sixteen",
        "seventeen",
        "eighteen",
        "nineteen",
    ]
    tens = [
        "",
        "",
        "twenty",
        "thirty",
        "forty",
        "fifty",
        "sixty",
        "seventy",
        "eighty",
        "ninety",
    ]

    if n < 10:
        return ones[n]

    if n < 20:
        return teens[n % 10]

    if n < 100:
        return tens[n // 10] + ("" if (n % 10 == 0) else "-" + ones[n % 10])


for i in range(20, 99):
    print(num_to_word(i))

# print(num_to_word(21))

twenty
twenty-one
twenty-two
twenty-three
twenty-four
twenty-five
twenty-six
twenty-seven
twenty-eight
twenty-nine
thirty
thirty-one
thirty-two
thirty-three
thirty-four
thirty-five
thirty-six
thirty-seven
thirty-eight
thirty-nine
forty
forty-one
forty-two
forty-three
forty-four
forty-five
forty-six
forty-seven
forty-eight
forty-nine
fifty
fifty-one
fifty-two
fifty-three
fifty-four
fifty-five
fifty-six
fifty-seven
fifty-eight
fifty-nine
sixty
sixty-one
sixty-two
sixty-three
sixty-four
sixty-five
sixty-six
sixty-seven
sixty-eight
sixty-nine
seventy
seventy-one
seventy-two
seventy-three
seventy-four
seventy-five
seventy-six
seventy-seven
seventy-eight
seventy-nine
eighty
eighty-one
eighty-two
eighty-three
eighty-four
eighty-five
eighty-six
eighty-seven
eighty-eight
eighty-nine
ninety
ninety-one
ninety-two
ninety-three
ninety-four
ninety-five
ninety-six
ninety-seven
ninety-eight


In FastAPI, implementing circuit breakers is useful for improving the resilience of your backend, especially when dealing with external services or APIs that might be unreliable. A circuit breaker helps prevent your application from repeatedly trying to call a failing service, which can cause cascading failures.

### Why Use Circuit Breakers?
- **Fail Fast:** Avoid long wait times by quickly returning errors when a service is down.
- **Stability:** Prevent system overload by stopping repeated calls to a failing service.
- **Graceful Degradation:** Allow your application to continue functioning in a limited capacity.

### How to Implement Circuit Breakers in FastAPI
You can use the **PyBreaker** library, which is a Python implementation of the circuit breaker pattern. It integrates well with FastAPI.

### Installation
```bash
pip install fastapi uvicorn requests pybreaker
```

### Basic Example Using PyBreaker

```python
from fastapi import FastAPI, HTTPException
import requests
from pybreaker import CircuitBreaker, CircuitBreakerError

app = FastAPI()

# Configure Circuit Breaker
breaker = CircuitBreaker(
    fail_max=3,  # Maximum number of failures before opening the circuit
    reset_timeout=10  # Time (in seconds) to wait before trying again
)

# Function wrapped with Circuit Breaker
@breaker
def call_external_service():
    response = requests.get("https://api.example.com/data")
    if response.status_code != 200:
        raise Exception("Service unavailable")
    return response.json()

@app.get("/data")
def get_data():
    try:
        data = call_external_service()
        return {"status": "success", "data": data}
    except CircuitBreakerError:
        raise HTTPException(status_code=503, detail="Service temporarily unavailable")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
```

### Explanation:
- `CircuitBreaker`: Monitors the success/failure of requests. After a certain number of failures, it "opens" the circuit.
- `fail_max=3`: After 3 failures, the circuit will open.
- `reset_timeout=10`: After 10 seconds, the circuit will move to a "half-open" state, allowing one request to check if the service is back.
- `CircuitBreakerError`: Raised when the circuit is open, indicating the service is temporarily unavailable.

### How It Works:
1. **Closed State**: The circuit is closed by default, and all requests go through.
2. **Open State**: If the request fails 3 times, the circuit opens, and all subsequent requests fail fast for the next 10 seconds.
3. **Half-Open State**: After 10 seconds, the circuit tries one request:
    - If successful, the circuit closes again.
    - If it fails, the circuit stays open.

### Run the FastAPI Server
```bash
uvicorn your_script_name:app --reload
```

### Why Use PyBreaker?
- **Simplicity**: Easy to integrate with existing FastAPI routes.
- **Customization**: You can adjust the failure threshold and timeout settings.

This approach is efficient for protecting your FastAPI backend from external service failures and ensuring a more resilient application.

In FastAPI, circular references usually occur when you have models or schemas that reference each other. This can lead to issues like import errors or RecursionError due to Python trying to resolve dependencies indefinitely.

### Common Scenario:
Imagine two Pydantic models where:
- Model A references Model B
- Model B references Model A

### Example of Circular Reference:
```python
# models/item.py
from pydantic import BaseModel
from models.user import User  # Circular import!

class Item(BaseModel):
    name: str
    owner: User
```

```python
# models/user.py
from pydantic import BaseModel
from models.item import Item  # Circular import!

class User(BaseModel):
    username: str
    items: list[Item]
```

In this example:
- `Item` references `User`, and `User` references `Item`.
- This creates a circular import and causes an error.

---

### Solutions:

#### 1. Use Type Hints with `ForwardRef`
You can use `ForwardRef` or string annotations to resolve this.

**Example:**
```python
# models/item.py
from pydantic import BaseModel
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from models.user import User  # Only imported for type checking

class Item(BaseModel):
    name: str
    owner: 'User'  # Use a string reference
```

```python
# models/user.py
from pydantic import BaseModel
from models.item import Item

class User(BaseModel):
    username: str
    items: list[Item]
```

- Using `'User'` as a string allows Python to delay the evaluation of the type.
- `TYPE_CHECKING` is used to avoid circular import while still getting type hints in IDEs.

---

#### 2. Move Imports Inside Functions
Move the import statement inside the function or method where it is needed.

**Example:**
```python
# models/item.py
from pydantic import BaseModel

class Item(BaseModel):
    name: str

    def get_owner(self):
        from models.user import User  # Local import avoids circular reference
        return User(username="JohnDoe")
```

---

#### 3. Restructure Your Code
If the circular dependency is complex, consider:
- **Merging related models** into a single file.
- **Creating a separate module** for shared or common types.

**Example:**
```python
# models/common.py
from pydantic import BaseModel

class ItemBase(BaseModel):
    name: str
```

```python
# models/item.py
from models.common import ItemBase

class Item(ItemBase):
    owner: 'User'
```

```python
# models/user.py
from models.common import ItemBase

class User(BaseModel):
    username: str
    items: list[ItemBase]
```

This reduces dependencies between modules.

---

### Best Practices:
- **Use ForwardRef or string annotations** when models reference each other.
- **Limit cross-references** by designing schemas carefully.
- **Refactor complex dependencies** into shared or common modules.

By organizing your models and using these techniques, you can avoid circular references and keep your FastAPI project maintainable and error-free.

Memory management in Python is handled automatically by the **Python memory manager** and **garbage collector**, but there are several tools available to optimize memory usage, debug memory leaks, and analyze performance.

## 🔹 **Key Python Tools for Memory Management**

### **1. `gc` (Garbage Collection Module)**
Python has **automatic garbage collection**, but you can manually control it with the `gc` module.

📌 **Usage:**
```python
import gc

# Force garbage collection
gc.collect()

# Check if garbage collection is enabled
print(gc.isenabled())

# Disable garbage collection (useful for performance testing)
gc.disable()
```
✅ **Best for:** Manually triggering garbage collection and debugging memory leaks.

---

### **2. `sys` (Memory Usage Monitoring)**
You can check the memory usage of Python objects with `sys.getsizeof()`.

📌 **Usage:**
```python
import sys

x = [1, 2, 3, 4, 5]
print(sys.getsizeof(x))  # Size in bytes
```
✅ **Best for:** Checking memory consumption of objects.

---

### **3. `tracemalloc` (Track Memory Usage & Leaks)**
`tracemalloc` is useful for identifying memory leaks by tracking memory allocations.

📌 **Usage:**
```python
import tracemalloc

tracemalloc.start()

# Code to analyze
x = [i for i in range(100000)]

# Snapshot memory usage
snapshot = tracemalloc.take_snapshot()
for stat in snapshot.statistics("lineno")[:10]:  # Top 10 memory-consuming lines
    print(stat)
```
✅ **Best for:** Debugging memory leaks and tracking memory usage by line number.

---

### **4. `memory_profiler` (Function-Level Profiling)**
A more detailed memory profiler that shows memory consumption per line in a function.

📌 **Installation:**
```bash
pip install memory_profiler
```

📌 **Usage:**
```python
from memory_profiler import profile

@profile
def test():
    x = [i for i in range(100000)]
    return x

test()
```
✅ **Best for:** Finding memory-heavy functions in your code.

---

### **5. `objgraph` (Visualizing Object References)**
If you suspect **circular references** or memory leaks, `objgraph` helps visualize object relationships.

📌 **Installation:**
```bash
pip install objgraph
```

📌 **Usage:**
```python
import objgraph

x = [1, 2, 3]
y = [x, x]  # Circular reference

objgraph.show_backrefs([y], filename='graph.png')
```
✅ **Best for:** Visualizing object references to detect memory leaks.

---

### **6. `psutil` (Monitor System-Level Memory Usage)**
You can monitor **RAM consumption** of your Python process using `psutil`.

📌 **Installation:**
```bash
pip install psutil
```

📌 **Usage:**
```python
import psutil

# Get memory usage of current Python process
process = psutil.Process()
print(f"Memory Usage: {process.memory_info().rss / (1024 * 1024)} MB")
```
✅ **Best for:** Checking real-time system memory usage of your script.

---

## 🔹 **Best Practices for Memory Management**
1. **Use Generators Instead of Lists** (avoid keeping large data in memory)
   ```python
   def my_generator():
       for i in range(1000000):
           yield i
   ```

2. **Delete Unused Objects** (`del` keyword)
   ```python
   x = [1, 2, 3]
   del x  # Remove from memory
   ```

3. **Avoid Circular References**
   ```python
   import weakref

   class A:
       pass

   a = A()
   a_ref = weakref.ref(a)  # Avoid strong reference cycles
   ```

4. **Optimize Data Structures**
   - Use `array` instead of lists for numeric data.
   - Use `__slots__` in classes to reduce memory footprint.

---

## 🔹 **Conclusion**
Python has built-in memory management, but tools like `gc`, `tracemalloc`, and `memory_profiler` can help **detect leaks, optimize memory usage, and debug performance issues**. 🚀

**Multi-tenant architecture** is a software architecture where a single instance of an application serves multiple tenants (clients or organizations). Each tenant's data is isolated and secure, but they share the same application and infrastructure resources. It's commonly used in **SaaS (Software as a Service)** applications to optimize resource usage and reduce costs. 

---

## 🔹 **Key Concepts of Multi-Tenant Architecture**

1. **Tenant:** An individual client or organization using the application.
2. **Isolation:** Each tenant's data is securely isolated from others.
3. **Shared Resources:** The application and infrastructure (e.g., servers, databases) are shared among tenants.
4. **Customization:** Tenants can have customized configurations without affecting others.

---

## 🔹 **Types of Multi-Tenant Architectures**

### 1. **Database-per-Tenant**
- **Description:** Each tenant has its own database.
- **Pros:**
  - Strong data isolation.
  - Easier data backup and recovery.
- **Cons:**
  - High operational cost if the number of tenants is large.
  - Resource overhead due to multiple databases.
- **Use Case:** When regulatory compliance or strict data isolation is required.

---

### 2. **Schema-per-Tenant**
- **Description:** A single database with a separate schema for each tenant.
- **Pros:**
  - Moderate isolation with shared resources.
  - Easier management compared to multiple databases.
- **Cons:**
  - Potential schema bloat with many tenants.
  - Complex queries when accessing multiple schemas.
- **Use Case:** Medium-sized applications requiring moderate isolation.

---

### 3. **Shared Schema with Tenant Identifier**
- **Description:** A single database and schema, but each record is tagged with a tenant identifier (e.g., `tenant_id` column).
- **Pros:**
  - Most efficient in terms of resource utilization.
  - Simplifies scaling and maintenance.
- **Cons:**
  - Weaker data isolation.
  - Risk of accidental data leaks if queries don't filter by `tenant_id`.
- **Use Case:** High-scale applications where cost efficiency is crucial.

---

## 🔹 **Design Considerations**
- **Data Isolation:** Choose a model that meets your security and compliance requirements.
- **Scalability:** Ensure the architecture can scale with the number of tenants.
- **Customization:** Allow configurations at the tenant level (e.g., themes, feature toggles).
- **Performance:** Optimize database queries to maintain performance across tenants.

---

## 🔹 **Implementing Multi-Tenant Architecture in FastAPI**

### **Example: Shared Schema with Tenant Identifier**

In FastAPI, you can implement multi-tenancy by adding a `tenant_id` to all database models and filtering requests based on the current tenant.

### **Step 1: Define the Models**
```python
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from database import Base

class Tenant(Base):
    __tablename__ = "tenants"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, unique=True, index=True)
    users = relationship("User", back_populates="tenant")

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    tenant_id = Column(Integer, ForeignKey("tenants.id"))
    tenant = relationship("Tenant", back_populates="users")
```

### **Step 2: Middleware for Tenant Identification**
Identify the tenant using subdomain or headers.

```python
from fastapi import FastAPI, Request

app = FastAPI()

@app.middleware("http")
async def add_tenant_to_request(request: Request, call_next):
    subdomain = request.url.hostname.split('.')[0]  # Get subdomain as tenant identifier
    request.state.tenant = subdomain
    response = await call_next(request)
    return response
```

### **Step 3: Dependency to Access Tenant-Specific Data**
```python
from sqlalchemy.orm import Session
from fastapi import Depends

def get_tenant_db(session: Session = Depends(get_db), request: Request = Depends()):
    tenant = request.state.tenant
    if not tenant:
        raise HTTPException(status_code=400, detail="Tenant not found")
    return session.query(Tenant).filter(Tenant.name == tenant).first()
```

### **Step 4: Querying Tenant-Specific Data**
```python
@app.get("/users/")
def read_users(tenant=Depends(get_tenant_db), db: Session = Depends(get_db)):
    users = db.query(User).filter(User.tenant_id == tenant.id).all()
    return users
```

---

## 🔹 **Best Practices**
- **Secure Tenant Isolation:** Always filter queries by `tenant_id` to prevent data leaks.
- **Performance Optimization:** Use database indexing on `tenant_id`.
- **Configuration Management:** Store tenant-specific configurations in a separate table.
- **Scalable Design:** Consider using a distributed database or sharding for high scalability.

---

## 🔹 **Pros and Cons of Multi-Tenant Architecture**
| Pros                                  | Cons                                     |
|---------------------------------------|------------------------------------------|
| Cost-effective resource usage          | Complexity in design and maintenance      |
| Easier updates and maintenance         | Weaker data isolation in shared schema    |
| Centralized management                 | Risk of noisy neighbors (resource contention) |
| Efficient scaling                      | Complex security and compliance requirements |

---

## 🔹 **Use Cases**
- **SaaS applications** with multiple clients (e.g., CRM, ERP systems).
- **Enterprise software** with separate workspaces for departments.
- **Educational platforms** serving multiple institutions.

---

## 🔹 **Conclusion**
Multi-tenant architecture is a powerful design pattern for **scalable SaaS applications**, balancing cost efficiency and data isolation. Choosing the right approach (`Database-per-Tenant`, `Schema-per-Tenant`, or `Shared Schema`) depends on your **security requirements, cost constraints, and scalability needs**.

**Docker Networking** enables communication between containers, services, and the external world. It allows containers to discover each other and communicate securely, making it a key component in **microservices architecture**.

---

## 🔹 **Types of Docker Networks**

1. **Bridge Network (Default)**
   - **Description:** Containers on the same bridge network can communicate using their container names.
   - **Use Case:** Single-host setups where containers need to communicate internally.
   - **Example:**
     ```sh
     docker network create my_bridge
     docker run --name container1 --network my_bridge alpine ping container2
     docker run --name container2 --network my_bridge alpine ping container1
     ```

---

2. **Host Network**
   - **Description:** The container shares the host's network stack. It directly uses the host's IP address.
   - **Use Case:** High-performance scenarios needing low network latency.
   - **Example:**
     ```sh
     docker run --network host nginx
     ```
   - **Note:** No isolation between container and host network, so ports can conflict.

---

3. **Overlay Network**
   - **Description:** Connects containers across multiple hosts, used in **Docker Swarm** or **Kubernetes**.
   - **Use Case:** Multi-host deployments for scalable microservices.
   - **Example:**
     ```sh
     docker network create -d overlay my_overlay
     docker service create --name web --network my_overlay nginx
     ```
   - **Note:** Requires a key-value store (like Consul, etcd) for managing cluster state.

---

4. **Macvlan Network**
   - **Description:** Assigns a unique MAC address to each container, making them appear as physical devices on the network.
   - **Use Case:** Situations requiring containers to be directly accessible on the physical network.
   - **Example:**
     ```sh
     docker network create -d macvlan \
       --subnet=192.168.1.0/24 \
       --gateway=192.168.1.1 \
       -o parent=eth0 macvlan_net
     docker run --network macvlan_net alpine ip a
     ```
   - **Note:** Advanced setup with specific network hardware requirements.

---

5. **None Network**
   - **Description:** No network interface is assigned to the container (except `lo`).
   - **Use Case:** Isolated containers with no network access.
   - **Example:**
     ```sh
     docker run --network none alpine ifconfig
     ```

---

## 🔹 **Default Networks in Docker**

- **bridge**: Default network for standalone containers on a single host.
- **host**: Shares the host's network namespace.
- **none**: No networking.
- **overlay**: Used in Docker Swarm mode for multi-host communication.
- **macvlan**: Direct connection to the physical network.

---

## 🔹 **Common Commands for Docker Networking**

### 1. **List Networks**
```sh
docker network ls
```

### 2. **Inspect a Network**
```sh
docker network inspect <network_name>
```

### 3. **Create a Network**
```sh
docker network create -d bridge my_bridge
```

### 4. **Connect a Container to a Network**
```sh
docker network connect my_bridge my_container
```

### 5. **Disconnect a Container from a Network**
```sh
docker network disconnect my_bridge my_container
```

### 6. **Remove a Network**
```sh
docker network rm my_bridge
```

---

## 🔹 **Docker Compose Networking**

In Docker Compose, services are automatically connected to a shared network. You can define custom networks as well.

### **Example: Docker Compose with Custom Network**
```yaml
version: '3'
services:
  app:
    image: my_app_image
    networks:
      - my_custom_network
  db:
    image: postgres
    networks:
      - my_custom_network

networks:
  my_custom_network:
    driver: bridge
```

### **Run with Docker Compose**
```sh
docker-compose up
```

- Containers can communicate using service names (`app` can ping `db`).

---

## 🔹 **Communication Between Containers**

- **Same Network:** Containers can communicate using their **container names** as hostnames.
- **Different Networks:** Containers need to be connected to each other's network using:
  ```sh
  docker network connect <network_name> <container_name>
  ```

---

## 🔹 **Port Mapping and Exposure**

- **Expose a Port:** Make a container's port accessible outside the host.
  ```sh
  docker run -p 8080:80 nginx
  ```
  - `-p 8080:80`: Maps host port `8080` to container port `80`.

- **Expose Multiple Ports:**
  ```sh
  docker run -p 8080:80 -p 8443:443 nginx
  ```

- **Expose a Range of Ports:**
  ```sh
  docker run -p 8000-8010:80 nginx
  ```

---

## 🔹 **DNS Resolution in Docker Networks**

- Docker uses an embedded **DNS server** to provide name resolution for containers.
- Containers can resolve each other by **container name** within the same custom network.
- In Docker Compose, services can resolve each other using **service names**.

---

## 🔹 **Network Security Tips**

- Use **Overlay Networks** for cross-host communication with encryption.
- Implement **Firewall rules** to control traffic between containers.
- Use **Network segmentation** to isolate sensitive services.
- Leverage **Docker secrets** for sensitive data (e.g., passwords, API keys).

---

## 🔹 **Troubleshooting Tips**

1. **Check Network Connectivity:**
   ```sh
   docker exec -it <container_name> ping <target_container_name>
   ```

2. **Check IP Address and Network Config:**
   ```sh
   docker inspect <container_name>
   ```

3. **Debug with Network Namespace:**
   ```sh
   docker run --rm -it --network container:<container_name> nicolaka/netshoot
   ```
   - `netshoot` is a powerful container image with networking tools like `curl`, `ping`, `dig`, etc.

---

## 🔹 **Best Practices**
- Use **Bridge Network** for single-host setups.
- Use **Overlay Network** for multi-host or cluster setups (e.g., Docker Swarm or Kubernetes).
- Prefer **Host Network** only for performance-critical applications.
- Isolate sensitive services using **Separate Networks**.
- Always **limit exposed ports** to minimize security risks.
- Utilize **Network aliases** for flexible container naming and communication.

---

## 🔹 **Conclusion**

- Docker networking is versatile and scalable, supporting various deployment models (single-host, multi-host, cloud-native).
- **Bridge** and **Overlay** networks are the most commonly used options.
- Security and isolation depend on the chosen network type and configuration.
- Mastering Docker networking is crucial for **microservices architecture** and **cloud-native applications**.